In [33]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

In [34]:
import requests
import pandas as pd
url = 'https://www.boligsiden.dk/propertyresult/getdata?searchId=b4fae611b9444fa88f6dd6cd5f39de8d&pageNumber=1&sortKey=12&sortDescending=false&displayTab=1&itemsPerPage=60'
response = requests.get(url)
response.status_code

200

In [35]:
response.json().keys()

dict_keys(['newSearchId', 'result', 'search', 'mapTokens'])

In [36]:
page = response.json()['result']

In [37]:
boligdf = pd.DataFrame(page['properties'])

In [38]:
boligdf = pd.DataFrame(page['properties'])

In [39]:
pd.options.display.max_columns = 100

In [40]:
totalcount = page['totalCount']
totalcount

64364

In [14]:
itemsperpage = 60

In [16]:
pages = round(totalcount/itemsperpage, 0)
pages

1073.0

In [53]:
def url_constructor():
    urls = []
    part_one = 'https://www.boligsiden.dk/propertyresult/getdata?searchId=b4fae611b9444fa88f6dd6cd5f39de8d&pageNumber='
    part_two = '&sortKey=12&sortDescending=false&displayTab=1&itemsPerPage=60'
    for i in range(1, int(pages)+1):
        urls.append(part_one + str(i) + part_two)
    return urls
        #print(i)

tests = url_constructor()

In [55]:
len(tests)

1073

In [30]:
import time
import random
import json
import tqdm

def scrape_append(url_list):
    bolig_df = pd.DataFrame()
    
    for url_link in tqdm.tqdm(url_list):
        time.sleep(0.5)
        response, call_id = connector.get(url_link,'DataScience_Boligsiden')
        Y = response.text
        Y_json = json.loads(Y)
        #data = Y_json['JobPositionPostings']
        pagedata = Y_json['result']
        data = pagedata['properties']
        df = pd.DataFrame(data)
        bolig_df = pd.concat([df, bolig_df], ignore_index=True)
    
    return bolig_df
url_input = url_constructor()
url_in_test = url_input[0:20]
housing_df = scrape_append(url_input)

100%|██████████| 1073/1073 [26:52<00:00,  1.64s/it] 


In [31]:
housing_df.shape

(64368, 39)

In [32]:
housing_df.to_csv('housing_data2.csv')

In [57]:
housing_df.head(30)

,id,itemTypeName,itemType,propertyLink,isFavorite,rating,priceDevelopment,hasOpenHouse,nextOpenHouse,nextOpenHouseSignup,imageLink300X200,energyMark,energyMarkLink,address,postal,city,paymentCash,downPayment,paymentExpenses,paymentGross,paymentNet,areaResidential,numberOfRooms,areaParcel,salesPeriod,redirectLink,openHouseRedirectLink,agentsLogoLink,financing,areaPaymentCash,areaWeighted,uniqueNumber,agentChainName,isArchive,dateRemoved,placeName,canShowSalesPeriodTotal,salesPeriodTotal,dateAnnounced
0,76a0489c20154d85b5a2b5f372d4be2e,Helårsgrund,700,~/salg/841209039,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/7...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Strandparken 28,8585,Glesborg,198.380,25.000,562,944,772,0,0,763,4.051,https://www.boligsiden.dk/viderestilling/76a04...,None,https://pic.boligsiden.dk/agent/23ed916c7b6840...,{'link': 'https://raadgivning.boligsiden.dk/bo...,260,-,841209039,danbolig,False,,Fjellerup,True,-,18-07-2008
1,0b52dadaab264002807a80236c487c8d,Fritidsgrund,800,~/salg/101775169,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/0...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,"Matildesvej, Prisgruppe 2",9670,Løgstør,195.000,25.000,125,916,748,0,0,1.200,4.060,https://www.boligsiden.dk/viderestilling/0b52d...,None,https://pic.boligsiden.dk/agent/8fd5118a5ac243...,{'link': 'https://raadgivning.boligsiden.dk/bo...,163,-,101775169,Asger Larsen,False,,,True,-,09-07-2008
2,edcd163674064d5686bbe9a098d3dc39,Fritidsgrund,800,~/salg/902405835,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/e...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,"Matildesvej, Prisgruppe 1",9670,Løgstør,225.000,25.000,42,1.067,871,0,0,1.200,4.060,https://www.boligsiden.dk/viderestilling/edcd1...,None,https://pic.boligsiden.dk/agent/8fd5118a5ac243...,{'link': 'https://raadgivning.boligsiden.dk/bo...,188,-,902405835,Asger Larsen,False,,,True,-,09-07-2008
3,f5b1322b94264ee9bab4081a1ba03687,Helårsgrund,700,~/salg/901685141,False,"{'ratings': {'conditionRating': None, 'kitchen...",-35%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/f...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Rasmus Larsens Vænge 1,8300,Odder,467.775,25.000,1.003,2.345,1.939,0,0,1.124,4.071,https://www.boligsiden.dk/viderestilling/f5b13...,None,https://pic.boligsiden.dk/agent/01f208c5b7b246...,{'link': 'https://raadgivning.boligsiden.dk/bo...,416,-,901685141,Nybolig,False,,,True,-,28-06-2008
4,01ab5948969c447284b7dad81f7942cf,Helårsgrund,700,~/salg/602404305,False,"{'ratings': {'conditionRating': None, 'kitchen...",-37%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/0...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Rasmus Larsens Vænge 2,8300,Odder,423.225,25.000,1.003,2.119,1.753,0,0,1.252,4.071,https://www.boligsiden.dk/viderestilling/01ab5...,None,https://pic.boligsiden.dk/agent/01f208c5b7b246...,{'link': 'https://raadgivning.boligsiden.dk/bo...,338,-,602404305,Nybolig,False,,,True,-,28-06-2008
5,4a907825bf6d48159e8ef16654731013,Helårsgrund,700,~/salg/820639161,False,"{'ratings': {'conditionRating': None, 'kitchen...",-36%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/4...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Rasmus Larsens Vænge 3,8300,Odder,445.500,25.000,1.003,2.229,1.843,0,0,980,4.071,https://www.boligsiden.dk/viderestilling/4a907...,None,https://pic.boligsiden.dk/agent/01f208c5b7b246...,{'link': 'https://raadgivning.boligsiden.dk/bo...,455,-,820639161,Nybolig,False,,,True,-,28-06-2008
6,531c816133074e478ffec08b8d30235a,Fritidsgrund,800,~/salg/463773799,False,"{'ratings': {'conditionRating': None, 'kitchen...",-62%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/5...

In [51]:
housing_df['rating'] = housing_df['rating'].astype(str)
housing_df['financing'] = housing_df['financing'].astype(str)
housing_distinct = housing_df.drop_duplicates()
housing_distinct.shape

(64362, 39)

In [50]:
#housing_distinct = housing_df.drop_duplicates()
#housing_distinct.shape
housing_df.dtypes

id                         object
itemTypeName               object
itemType                    int64
propertyLink               object
isFavorite                   bool
rating                     object
priceDevelopment           object
hasOpenHouse                 bool
nextOpenHouse              object
nextOpenHouseSignup          bool
imageLink300X200           object
energyMark                 object
energyMarkLink             object
address                    object
postal                     object
city                       object
paymentCash                object
downPayment                object
paymentExpenses            object
paymentGross               object
paymentNet                 object
areaResidential            object
numberOfRooms              object
areaParcel                 object
salesPeriod                object
redirectLink               object
openHouseRedirectLink      object
agentsLogoLink             object
financing                  object
areaPaymentCas

In [56]:
housing_distinct.to_csv('housing_data_noduplicates.csv')

In [98]:
howlong = 'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?poitypes='
list_of_dests = ['doctor', 'daycare','hospital','school','stop','supermarket','train','library','pharmacy']
#adress = '&fromaddress=Ahornsgade%2016,%202200%20K%C3%B8benhavn%20N'
adress_format = '&fromaddress='
streetname = 'Ahornsgade'
m1 = '%20'
house_no = '16'
m2 = ',%20'
post_no = '2200'
m3 = '%20'
#city ='K%C3%B8benhavn' 
city ='Købenbenhavn' 
m4 = '%20N'
accestoken = '&mot=foot&token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInByaXZzIjogInIxWjByMEYwazZCdFdxUWNPVXlrQi95NlNVcEp2MlFiZ3lYZXRxNEhZNFhPLzNZclcwK0s5dz09In0.fP4JWis69HmaSg5jVHiK8nemiCu6VaMULSGGJyK4D4PkWq4iA1+nSHWMaHxepKwJ83sEiy9nMNZhv7BcktRNrA'
logfile2 = 'how_long_log.csv'
connector = scraping_class.Connector(logfile2)

In [99]:
response_hl = requests.get(howlong+list_of_dests[0] + adress_format + streetname+ m1 + house_no + m2 + post_no + m3 + city + m4 + accestoken)
response_hl.text

'{"doctor": {"poi_id": 322295, "name": "Karina M\\u00f8ller", "fromlatlng": [55.69181676, 12.55912262], "travelseconds": 166.6512, "mot": "foot", "routepolyline": "qh|rIe~skAnAtBjAmDTs@\\\\cAJWa@q@??", "euclideanmeters": 123.94, "tolatlng": [55.69092727, 12.56030691], "routedmeters": 231.46}}'

In [138]:
import string

def get_houseno(adress_in):
    if 'Prisgruppe' in str(adress_in):
        return 'null'
    elif ',' in str(adress_in):
        bd = adress_in.split(',')
        bd = bd[0].split()
        if bd[-1] in string.ascii_uppercase:
            return bd[-2]
        else: 
            return bd[-1]
    else:
        ad = adress_in.split()
        if len(ad) > 1:
            return ad[-1]
        else:
            return 'null'
get_houseno('Buddinge Hovedgade 78 E, 1.')

'78'

In [139]:
housing_distinct['house_no'] = housing_distinct['address'].apply(get_houseno)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [171]:
def distance_features(adresse_input, id_data):
    list_of_dests = ['doctor', 'daycare','hospital','school','stop','supermarket','train','library','pharmacy']
    COL = [[housing_distinct.columns], list_of_dests]
    collection_distances = pd.DataFrame(columns = COL)
    for i in list_of_dests:
        collection_distances[i] = howlong + i + adress_format + adress + accestoken
        collection_distances['id'] = id_data
distance_features('Buddinge Hovedgade 78 E, 1.' , '666')

ValueError: all arrays must be same length

In [179]:
logfile3 = 'how_long_log.csv'
connector = scraping_class.Connector(logfile3)
url_for_coordinates = 'https://dawa.aws.dk/postnumre?nr=8600'
response_coordinates = requests.get(url_for_coordinates)
output = response_coordinates.json()
output = output[0]

In [199]:
i, e = output['visueltcenter'][0], output['visueltcenter'][1]

In [215]:
logfile3 = 'how_long_log.csv'
connector = scraping_class.Connector(logfile3)
import time

def post_coordinates(post_no):
    time.sleep(0.5)
    print(post_no)
    response_coordinates = requests.get('https://dawa.aws.dk/postnumre?nr=' + post_no)
    output = response_coordinates.json()
    output = output[0]
    return output['visueltcenter']

In [216]:
housing_distinct['Long_Lat'] = housing_distinct['postal'].apply(post_coordinates)

8585
9670
9670
8300
8300
8300
9560
7680
6720
4970
8500
9830
6940
4673
7990
4873
6470
7884
6853
9850
4733
9800
9800
9800
9640
6920
7870
6971
5620
7673
6650
7280
6893
6705
4591
4970
7741
7280
4874
4250
7760
7673
6950
6950
6950
6950
6950
6950
9640
7770
7200
7200
9560
7200
9990
6950
6440
7570
7700
6430
8950
7860
6792
4930
6270
6270
6792
7620
9480
6950
6971
9560
4653
9670
9560
9640
9640
9640
9640
9640
9640
9640
9640
9640
9640
9640
9640
9990
8500
4983
6500
7884
2450
7100
6920
9850
9460
9640
4912
6430
8400
4673
4673
4673
7755
9492
4891
8350
6800
3730
4733
9982
6430
3700
6230
7900
9940
7160
7990
9480
7830
6990
4171
9640
4970
4970
7200
4874
6520
4900
6818
6753
6960
9520
6940
7830
9600
4100
8300
8300
8300
8300
8300
9760
7900
4760
5683
3730
6330
6950
8305
9690
9480
5610
4250
7600
4970
6440
7760
7790
4900
4850
7742
7600
7884
4840
6470
9830
6094
6950
6000
6000
6950
6000
8420
9690
9850
6000
5500
6920
6920
9850
4900
9850
9850
9850
5900
9690
9280
9574
6670
8300
7620
6920
7900
9560
6720
4874
4970
7700


4874
9990
7080
5900
5600
4990
9300
7300
9370
9940
9800
4990
5464
5464
6830
5892
5610
7300
7183
6200
9280
9800
5270
6092
5400
6830
8882
9370
5250
9460
7840
9500
6430
9670
8400
4100
6500
9850
8600
8653
5683
6830
4573
9800
9800
4700
7752
9800
9800
9800
9800
9800
7650
8544
8740
8620
6760
4900
7700
9800
8300
6200
8300
6740
7140
8722
9560
9981
9982
9500
7700
9493
6440
5642
8970
9370
8832
6720
9560
9300
9560
5960
9700
4930
9640
6100
6100
6430
7361
9480
7080
8585
7600
6990
6682
4660
6310
6990
9560
5580
7870
7000
5935
9640
7700
7441
9940
8585
9940
6094
6100
4591
9940
9480
9260
5400
3730
9370
8990
9800
9940
7900
7361
6940
7830
7130
7790
6830
7490
6900
3730
8600
7790
5500
6100
9881
5932
5600
9881
6830
5610
9800
4200
4800
4572
6630
9800
9560
4683
7730
7730
6780
7730
7730
9940
7730
7730
9940
5900
9940
7730
7730
4900
8410
9940
6792
9940
6880
7730
6771
6310
4873
9370
6470
7200
6857
7730
4281
5932
4690
5610
4970
5700
5450
7870
9850
9850
5672
6950
7870
4970
8585
7950
5700
5700
8420
5932
4573
4573
4900


9300
9500
9870
7870
5000
9460
9881
9881
4970
6580
8444
5610
8850
6823
6500
3100
8550
7860
8643
6971
9800
9560
6310
9690
4873
4390
7840
9480
6830
7700
8585
9480
7770
9800
3120
6270
9490
6310
8981
8920
6520
8963
9900
9370
4970
6200
9460
6990
4874
7760
9982
8981
6270
6100
5881
8981
3400
7870
3400
5620
3400
6900
3400
3400
8930
3400
8960
7400
3400
8930
4780
8585
4690
4690
4690
4690
4970
9850
8752
4690
4690
4690
8752
8940
9690
7150
7080
8400
8550
7183
8850
7790
6980
7600
9560
6900
6990
9300
6070
5370
7770
9940
8970
6960
9900
9490
7323
5892
9870
7600
4593
4873
4593
4270
6792
9560
6430
9800
9370
7080
9480
6900
5700
4850
9370
9982
6792
8400
6200
7673
7400
8920
7400
7400
8783
7760
8960
7870
7400
4874
9981
6200
6792
8930
6100
9382
6520
8930
6300
7760
6064
6270
8850
9900
9370
9990
6880
8585
4632
4500
4793
6973
6760
8700
9900
9300
6240
7700
8585
5600
9940
6650
5932
7800
8586
7323
6960
9800
4400
5953
4872
4690
4690
7620
7600
9800
7860
8882
5700
8300
6430
9370
6753
4684
9700
9850
7130
4684
8700
4684


9760
6800
5672
4791
6270
4913
8600
7200
7080
4942
7760
7323
9480
7741
5592
5932
8700
4760
6893
9881
7400
4220
7120
6300
8300
6990
4622
6052
6270
6990
6000
5610
9480
6340
8600
5883
5953
9300
8600
8600
8832
8600
8600
7270
8600
6893
6792
6623
4920
8740
9480
4943
9460
4873
5600
4070
8400
4970
9480
4970
5600
4300
4772
4200
8420
8305
6900
8305
8305
7830
8305
4750
8305
8305
8305
9460
6470
5270
5491
6372
6100
9800
6200
6261
9560
9382
6200
5800
4230
6792
6100
7400
9493
7100
6560
9500
9370
6440
4500
4281
9480
6990
4872
7570
6893
6990
4850
6990
6720
7500
7184
8400
9900
9500
9670
8800
9670
4190
5750
8382
4592
9670
6000
4173
9670
7620
9670
9670
5874
9670
7800
9940
9490
8541
9981
9300
5700
5485
5600
9490
9370
4470
4900
6430
6670
7790
9490
5600
6950
9610
6950
7570
3790
6950
7500
5853
6100
3250
7620
9240
9240
9240
4900
8400
9280
9240
9240
4520
9240
9240
3630
6000
7550
8722
9670
9480
4930
8420
9541
5772
9480
4571
8832
5871
6800
5800
4571
7840
4243
9480
9480
6840
9940
4500
9670
7673
4791
9480
9480
6731


6960
7171
6100
9900
6100
6100
6100
4690
8961
6830
7130
9690
6280
4500
6100
4500
6100
7130
6100
8586
8500
6100
5683
7100
4400
6990
4863
6100
6100
6100
8653
4983
4400
8500
6950
6960
9230
9370
7430
9370
7000
9870
7000
7000
7182
4900
2630
7673
9670
7870
7870
7323
7600
6430
7600
7600
7600
6900
7080
8300
5631
7500
7600
5881
5800
4900
6261
7400
5463
4970
7870
2150
7400
7400
2150
9940
4895
6900
5400
7900
9230
9230
9230
9230
2150
8783
8420
2150
7361
5953
3630
4792
8660
8961
6720
4400
6792
9940
4930
9280
6340
6893
6800
4060
6340
7700
9560
8850
7441
7100
6100
7130
9670
5560
7990
4470
9982
9670
7130
3700
5464
6900
9640
9700
7130
6330
6100
4180
4400
7700
5800
6920
7442
7600
5900
6854
6854
9640
6100
7870
6470
6440
9440
5600
9640
4500
7600
9850
9230
9460
5750
7870
6792
9560
6753
6800
7500
8740
5591
7884
7100
9480
5985
7323
6240
9900
6430
9480
8961
3770
5892
6760
9881
9690
4793
5792
7400
7000
7620
7760
8420
6270
6240
5540
9300
8660
9560
9370
9560
6230
4050
4760
7323
8370
4550
9370
6792
9560
9760
5853


...

5700
3700
8620
8000
9493
1310
2942
8544
3630
4600
2000
4000
7000
9990
5000
4060
8000
9800
8700
8660
9480
8930
5750
2620
8000
4540
2300
4600
4800
9800
4800
8900
2450
9000
6700
3060
5610
8000
2840
3100
2600
5700
7400
4400
4873
2791
8320
5220
2000
4340
9620
4683
2700
8882
8300
8870
6710
5230
4581
1401
9000
4534
7600
8740
9670
2300
9620
8520
7900
5270
4400
6630
4300
8270
8860
4652
5230
8270
9560
4400
6000
2600
2670
3100
5853
6800
9210
8000
8751
5953
6800
8550
6715
8381
8464
2100
7700
9000
4450
4300
7120
4583
9700
3500
2610
8270
7480
3650
9620
4573
2100
4941
2650
8660
8500
5270
3730
4000
1307
2635
3751
7840
9560
4683
6823
9300
3650
3400
8355
2770
2670
2400
9000
4560
1601
7480
8000
4671
8990
4591
2200
3050
2820
6823
2900
1462
3400
6920
2500
5270
4320
2860
4200
4420
2610
8990
2720
8800
4000
4942
7451
2200
8660
5683
2650
9881
7451
2720
3500
9370
2980
8800
8544
9300
7840
8520
7140
8250
8900
8766
2000
8830
3740
4600
4683
9550
4000
9000
5900
4300
4440
8660
8200
9200
2610
8830
9200
4591
7700
3782


4220
2605
4700
5600
7752
8585
5250
2300
4600
9000
3400
6740
9280
9310
7000
3100
2300
2830
8850
8361
7100
9362
7323
5560
6000
9700
8382
2830
9000
7400
2970
6000
4720
4300
2830
4683
5620
6000
6400
2840
8600
6040
3000
8200
1818
4560
5700
3230
5230
5600
8380
4030
9300
2300
4760
7441
4060
6600
8680
7673
8300
7120
5874
4573
9990
5500
5210
8320
7190
5600
4100
2300
2300
2150
4591
8300
2300
2750
8200
2920
9240
8000
5240
2300
2300
2300
9000
9340
7451
9990
5700
2300
8444
4700
8800
8380
2840
7182
2610
8800
3670
8544
8961
6510
8382
8660
5540
2000
4000
2300
2300
4600
8520
8400
8620
4700
7620
8000
3050
7500
6700
8981
3320
9000
2300
6000
2700
8740
5900
2300
5300
6893
8660
2610
3480
9370
8600
2765
9260
8200
2765
8560
8300
3210
4540
5800
9510
6000
4500
4700
8250
9293
3520
8464
2930
4850
7900
2600
9400
5580
8700
3070
7800
7500
5600
6740
8600
2300
7200
8000
7970
3700
3660
2920
2640
3550
4700
9000
6920
4600
9370
4540
9280
7000
8471
2680
9220
5800
8471
3500
2820
7840
2300
2635
5884
2300
1318
6094
8570
4912


4780
2600
1959
8240
7840
5610
3500
7120
2100
2300
2100
5762
1311
8500
3210
3670
8560
9400
2100
7700
2300
2100
2200
4000
1458
2200
2300
4174
5683
9670
6580
1818
5000
2300
4270
2650
2770
2300
8300
6950
4800
5500
6900
8500
6857
9280
6840
9700
4230
6830
4622
8000
2950
4220
3360
8723
2450
9352
4440
8660
4653
8250
2920
8960
4490
9280
9493
4000
8210
9280
4000
2720
9000
6600
6330
9400
3000
6510
6960
9000
4000
2000
5220
7100
2820
5250
9280
7730
6700
2880
4970
6950
4671
2720
8382
3460
9280
8500
8700
9280
2900
8766
2920
2630
2730
2690
4040
9000
7160
8722
5700
8700
6100
9280
8500
5672
8850
2720
8800
9492
2770
4040
4874
3450
7441
2680
8600
4690
5260
8362
2100
9300
8920
9240
6580
4623
7400
4030
9280
2860
4760
4293
9670
7323
2670
4040
2300
8220
6760
8850
9280
8420
4100
2870
4970
4200
4700
6760
3070
5220
1601
3000
7770
2791
9280
4960
5683
8305
8700
9280
9280
4672
2650
9700
6752
8382
1669
3060
6000
8660
5000
7600
5800
7000
7100
3150
2720
9280
8000
3500
2650
2100
4900
5771
4030
3060
8585
2610
6270
2665


9300
2680
4281
5220
3310
2400
5772
4050
2640
5330
9800
5854
5270
9000
3520
4220
4600
2300
9280
4420
7900
2000
3000
6720
2605
5210
6705
2640
7100
5230
4200
5953
6857
2500
8600
3390
8361
3400
8752
3460
8751
1669
7400
9300
8240
8700
9210
7480
8000
9681
3700
9990
4200
3200
4171
9800
8700
9681
8400
5900
9000
9830
8800
3770
2830
6580
4200
4735
1620
6200
5330
5960
4340
8300
5800
6700
1568
8300
4970
8000
7840
4534
3480
9690
4750
7400
8766
3520
8700
4600
4700
2500
2630
2900
8600
8783
4873
8930
3500
4320
7800
6230
6705
3460
9320
8600
5700
2970
5500
2840
7250
6960
4300
2720
6400
7990
8660
8585
8000
5800
4880
7000
5750
7860
6700
5250
8700
2650
2920
4040
5450
5856
2610
3140
4050
2200
7000
8000
3060
5580
8723
2960
2750
8000
2970
2500
4873
2860
5270
8382
8600
6960
4550
4690
9430
4780
9000
9640
9210
2920
2600
8361
2200
4700
8500
6705
8960
2300
9940
4550
2700
4100
2630
8000
4573
8660
5000
2610
9740
4873
4600
4270
4970
4690
4550
9982
3520
3600
8000
9400
2100
9000
2860
2930
8362
9500
2400
3630
9230
9610


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [212]:
post_coordinates('2880')

[12.44587672, 55.76191781]

In [217]:
housing_distinct.to_csv('housing_data_wcoordinates.csv')

In [218]:
housing_distinct.head()

,id,itemTypeName,itemType,propertyLink,isFavorite,rating,priceDevelopment,hasOpenHouse,nextOpenHouse,nextOpenHouseSignup,imageLink300X200,energyMark,energyMarkLink,address,postal,city,paymentCash,downPayment,paymentExpenses,paymentGross,paymentNet,areaResidential,numberOfRooms,areaParcel,salesPeriod,redirectLink,openHouseRedirectLink,agentsLogoLink,financing,areaPaymentCash,areaWeighted,uniqueNumber,agentChainName,isArchive,dateRemoved,placeName,canShowSalesPeriodTotal,salesPeriodTotal,dateAnnounced,house_no,Long_Lat
0,76a0489c20154d85b5a2b5f372d4be2e,Helårsgrund,700,~/salg/841209039,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/7...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Strandparken 28,8585,Glesborg,198.380,25.000,562,944,772,0,0,763,4.051,https://www.boligsiden.dk/viderestilling/76a04...,None,https://pic.boligsiden.dk/agent/23ed916c7b6840...,{'link': 'https://raadgivning.boligsiden.dk/bo...,260,-,841209039,danbolig,False,,Fjellerup,True,-,18-07-2008,28,"[10.64766235, 56.58757292]"
1,0b52dadaab264002807a80236c487c8d,Fritidsgrund,800,~/salg/101775169,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/0...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,"Matildesvej, Prisgruppe 2",9670,Løgstør,195.000,25.000,125,916,748,0,0,1.200,4.060,https://www.boligsiden.dk/viderestilling/0b52d...,None,https://pic.boligsiden.dk/agent/8fd5118a5ac243...,{'link': 'https://raadgivning.boligsiden.dk/bo...,163,-,101775169,Asger Larsen,False,,,True,-,09-07-2008,null,"[9.30949603, 56.95819912]"
2,edcd163674064d5686bbe9a098d3dc39,Fritidsgrund,800,~/salg/902405835,False,"{'ratings': {'conditionRating': None, 'kitchen...",0%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/e...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,"Matildesvej, Prisgruppe 1",9670,Løgstør,225.000,25.000,42,1.067,871,0,0,1.200,4.060,https://www.boligsiden.dk/viderestilling/edcd1...,None,https://pic.boligsiden.dk/agent/8fd5118a5ac243...,{'link': 'https://raadgivning.boligsiden.dk/bo...,188,-,902405835,Asger Larsen,False,,,True,-,09-07-2008,null,"[9.30949603, 56.95819912]"
3,f5b1322b94264ee9bab4081a1ba03687,Helårsgrund,700,~/salg/901685141,False,"{'ratings': {'conditionRating': None, 'kitchen...",-35%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/f...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Rasmus Larsens Vænge 1,8300,Odder,467.775,25.000,1.003,2.345,1.939,0,0,1.124,4.071,https://www.boligsiden.dk/viderestilling/f5b13...,None,https://pic.boligsiden.dk/agent/01f208c5b7b246...,{'link': 'https://raadgivning.boligsiden.dk/bo...,416,-,901685141,Nybolig,False,,,True,-,28-06-2008,1,"[10.24464778, 55.92457284]"
4,01ab5948969c447284b7dad81f7942cf,Helårsgrund,700,~/salg/602404305,False,"{'ratings': {'conditionRating': None, 'kitchen...",-37%,False,01. jan. 00:00,False,https://pic.boligsiden.dk/property/300x200/1/0...,,https://sparenergi.dk/forbruger/vaerktoejer/de...,Rasmus Larsens Vænge 2,8300,Odder,423.225,25.000,1.003,2.119,1.753,0,0,1.252,4.071,https://www.boligsiden.dk/viderestilling/01ab5...,None,https://pic.boligsiden.dk/agent/01f208c5b7b246...,{'link': 'https://raadgivning.boligsiden.dk/bo...,338,-,602404305,Nybolig,False,,,True,-,28-06-2008,2,"[10.24464778, 55.92457284]"
